# Purpose 1: Combine Data

Currently, I have one Excel workbook with various sheets of Web of Science records. The code below easily combines all sheets into one sheet for ease of use.

In [ ]:
import pandas as pd
import xlrd

In [8]:
df = pd.concat(pd.read_excel('Full_WoS_Sample.xlsx',sheet_name=None), ignore_index=True)

In [9]:
df

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,J,"Glidden, CK; Field, LC; Bachhuber, S; Hennesse...",NaN,NaN,NaN,"Glidden, Caroline K.; Field, Laurel C.; Bachhu...",NaN,NaN,Strategies for managing marine disease,ECOLOGICAL APPLICATIONS,...,NaN,NaN,NaN,NaN,"Green Submitted, hybrid",NaN,NaN,NaN,WOS:000828498100001,View Full Record in Web of Science
1,J,"Avery-Gomm, S; Borrelle, SB; Provencher, JF",NaN,NaN,NaN,"Avery-Gomm, Stephanie; Borrelle, Stephanie B.;...",NaN,NaN,Linking plastic ingestion research with marine...,SCIENCE OF THE TOTAL ENVIRONMENT,...,NaN,NaN,NaN,NaN,Green Submitted,NaN,NaN,NaN,WOS:000436605400143,View Full Record in Web of Science
2,J,"Guerra, AS",NaN,NaN,NaN,"Guerra, Ana Sofia",NaN,NaN,Wolves of the Sea: Managing human-wildlife con...,MARINE POLICY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000454467200045,View Full Record in Web of Science
3,J,"Halliday, WD; Dawson, J; Yurkowski, DJ; Doniol...",NaN,NaN,NaN,"Halliday, William D.; Dawson, Jackie; Yurkowsk...",NaN,NaN,Vessel risks to marine wildlife in the Talluru...,ENVIRONMENTAL SCIENCE & POLICY,...,NaN,NaN,NaN,NaN,"Green Published, hybrid",NaN,NaN,NaN,WOS:000723249900001,View Full Record in Web of Science
4,J,"Samhouri, JF; Feist, BE; Fisher, MC; Liu, O; W...",NaN,NaN,NaN,"Samhouri, Jameal F.; Feist, Blake E.; Fisher, ...",NaN,NaN,Marine heatwave challenges solutions to human-...,PROCEEDINGS OF THE ROYAL SOCIETY B-BIOLOGICAL ...,...,NaN,NaN,NaN,NaN,"Green Published, hybrid",NaN,NaN,NaN,WOS:000724057500007,View Full Record in Web of Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,J,"Wang, RL; Jiang, CX; Huang, TT; Zhang, Z; Wang...",NaN,NaN,NaN,"Wang, Rulin; Jiang, Chunxian; Huang, Tingting;...",NaN,NaN,A Simulation Study of the Geographical Distrib...,POLISH JOURNAL OF ENVIRONMENTAL STUDIES,...,NaN,NaN,NaN,NaN,gold,NaN,NaN,NaN,WOS:000513486600034,View Full Record in Web of Science
49996,J,"Shapouri, MRSA; Mahmoodi, P; Najafabadi, MG; H...",NaN,NaN,NaN,"Shapouri, Masoud Reza Seyfi Abad; Mahmoodi, Pe...",NaN,NaN,A novel competitive ELISA for detection of ant...,VETERINARY RESEARCH FORUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000880674100014,View Full Record in Web of Science
49997,J,"Brown, G; Rhodes, J; Lunney, D; Goldingay, R; ...",NaN,NaN,NaN,"Brown, Greg; Rhodes, Jonathan; Lunney, Daniel;...",NaN,NaN,The influence of sampling design on spatial da...,TRANSACTIONS IN GIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000477297000001,View Full Record in Web of Science
49998,J,"Castro, KC; Leblond, M; Cote, SD",NaN,NaN,NaN,"Castro, Karina Charest; Leblond, Mathieu; Cote...",NaN,NaN,Costs and benefits of post-weaning association...,BEHAVIOUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000430341100004,View Full Record in Web of Science


This seems to have worked, so now I'll save to a csv file

In [10]:
df.to_csv('Combined_WoS_Dataset.csv')

# Purpose 2: Prepare Data For Sampling

Sampling will involve dividing articles by where the first author resided at the time of publication into US and non-US groups. This involves using regex code to pull just the first author's address out of the address data provided for all authors by WoS.

In [ ]:
address_list=list(df['Addresses'])
address_list[:3]

In [ ]:
address_strings=[]

for item in address_list:  #Make everything in the column  a string of text
    string=str(item)
    address_strings.append(string)

## Using pycountry to create a list of expected countries 

In [29]:
import pycountry

In [31]:
countries=[]
for string in address_strings:
    for country in pycountry.countries:
        if country.name in string:  ##Country name in pycountry is United States, not USA like Web of Science uses
            countries.append(country.name)

In [33]:
unique_countries=set(countries) ##Removes duplicates
print(len(unique_countries))



189


## Regex Method for Gathering First Author Address Data

In [13]:
import re

In [14]:
country_regex=r'[a-zA-Z ]+;|[a-zA-Z ]+$'

In [23]:

#finding regex matches
match_list=list()

for address in address_strings:
    match=re.findall(country_regex,address)
    match_list.append(match)

#Cleaning the list of regex matches
list_of_matches=list()

for matches in match_list:
    in_list=matches
    out_list= [re.sub(';', '', item) for item in in_list]
    
    list_of_matches.append(out_list)

In [24]:
#Checking to see if length of records is still correct
len(list_of_matches)

50000

In [25]:
list_of_matches[1]

[' Australia', ' Australia', ' New Zealand', ' Canada']

In [26]:
#Remove leading spaces
##Fixing spaces on all entries
clean_countries=[[country.lstrip() for country in matches] for matches in list_of_matches]
len(clean_countries) #length is correct

50000

In [27]:
clean_countries[100:110]

[['Spain', 'Spain', 'Arthur', 'Portugal', 'Kenya', 'Australia', 'Norway'],
 ['Canada', 'Canada', 'Canada'],
 ['Antonio', 'England'],
 ['Georgia', 'Greece'],
 ['USA', 'USA'],
 ['Daniel',
  'Gustav',
  'Marcus',
  'Sweden',
  'Giovanni',
  'Hung',
  'Australia',
  'USA',
  'Czech Republic',
  'Sweden',
  'England',
  'Marcus',
  'USA',
  'Australia',
  'Australia',
  'Italy',
  'Australia',
  'England'],
 ['Kenya', 'Norway', 'Germany', 'Kenya'],
 ['Alemayehu', 'Ethiopia', 'England'],
 ['South Africa', 'South Africa', 'South Africa'],
 ['Adolfo', 'Spain', 'Adolfo', 'Samir', 'Cape Verde', 'Cape Verde', 'Wales']]

In [43]:
#Just based on the regex, sometimes names are grabbed in addition to country of residence

##Next I need to get rid of the names still present and collect just the first author's country
    
first_author_country=list()
for address in clean_countries:
    
    if len(address)==1:
        first_author_country.append(address[0])
        
       
    else: 
        found_country = False
        for idx,potential_country in enumerate(address):
            if potential_country in unique_countries:
                first_author_country.append(potential_country)
                found_country= True
                
                break
                
        #This code handles countries that appeared in WoS data that weren't in pycountries
        
        #Current code appends just 'USA' because all missing countries have been corrected
        #The only error left is when there is a state abbreiviation in front of USA in an address
        if not found_country :
            first_author_country.append('USA')
#             first_author_country.append('Country not found')
#             print('Didnt find country in '+str(address))

In [42]:
#Here are some countries that weren't included in pycountry, but are in the WoS data
#This was found thorugh the 'Didn't find country in...' command
countries_to_add=['England','Taiwan','Scotland','Iran','Peoples R China','USA','Czech Republic','North Ireland','South Korea',
'Vietnam','Russia','Tanzania','U Arab Emirates','Wales','Rep Congo','Bolivia','DEM REP CONGO','Trinidad Tobago',
 'Papua N Guinea','Venezuela']

for country in countries_to_add:
    unique_countries.add(country)

In [44]:
len(first_author_country)

50000

In [46]:
first_author_country[:10]

['USA',
 'Australia',
 'USA',
 'Canada',
 'USA',
 'Scotland',
 'USA',
 'Kenya',
 'USA',
 'India']

# Inputing geographic data back into dataset

In [49]:
df=pd.read_csv('Combined_WoS_Dataset.csv')

C:\Users\12622\AppData\Local\Temp\ipykernel_15144\4258133060.py:1: DtypeWarning: Columns (11,12,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('Combined_WoS_Dataset.csv')


In [50]:
df

,Authors,Author Full Names,Article Title,Source Title,Author Keywords,Keywords Plus,Abstract,Addresses,Affiliations,Reprint Addresses,...,"Times Cited, WoS Core","Times Cited, All Databases",Publisher,ISSN,eISSN,Publication Year,DOI,DOI Link,Open Access Designations,Highly Cited Status
0,"Glidden, CK; Field, LC; Bachhuber, S; Hennesse...","Glidden, Caroline K.; Field, Laurel C.; Bachhu...",Strategies for managing marine disease,ECOLOGICAL APPLICATIONS,disease ecology; marine conservation; marine w...,ACUTE RESPIRATORY SYNDROME; ENDANGERED WHITE A...,The incidence of emerging infectious diseases ...,"[Glidden, Caroline K.; Field, Laurel C.; Bachh...",NaN,"Gravem, SA (corresponding author), Oregon Stat...",...,1,1,WILEY,1051-0761,1939-5582,2022.0,10.1002/eap.2643,http://dx.doi.org/10.1002/eap.2643,"Green Submitted, hybrid",NaN
1,"Avery-Gomm, S; Borrelle, SB; Provencher, JF","Avery-Gomm, Stephanie; Borrelle, Stephanie B.;...",Linking plastic ingestion research with marine...,SCIENCE OF THE TOTAL ENVIRONMENT,Plastic pollution; Conservation; Policy; Waste...,DEBRIS; MANAGEMENT; SEABIRDS; MICROPLASTICS; S...,Plastic is an increasingly pervasive marine po...,"[Avery-Gomm, Stephanie] Univ Queensland, Sch B...",NaN,"Avery-Gomm, S (corresponding author), Univ Que...",...,26,26,ELSEVIER SCIENCE BV,0048-9697,1879-1026,2018.0,10.1016/j.scitotenv.2018.04.409,http://dx.doi.org/10.1016/j.scitotenv.2018.04.409,Green Submitted,NaN
2,"Guerra, AS","Guerra, Ana Sofia",Wolves of the Sea: Managing human-wildlife con...,MARINE POLICY,Competition; Depredation; Human-wildlife confl...,CONSERVATION; MARINE; DEPREDATION; FISHERIES; ...,Human-wildlife conflict has been receiving inc...,"[Guerra, Ana Sofia] Univ Calif Santa Barbara, ...",NaN,"Guerra, AS (corresponding author), Univ Calif ...",...,28,29,ELSEVIER SCI LTD,0308-597X,1872-9460,2019.0,10.1016/j.marpol.2018.11.002,http://dx.doi.org/10.1016/j.marpol.2018.11.002,NaN,NaN
3,"Halliday, WD; Dawson, J; Yurkowski, DJ; Doniol...","Halliday, William D.; Dawson, Jackie; Yurkowsk...",Vessel risks to marine wildlife in the Talluru...,ENVIRONMENTAL SCIENCE & POLICY,Arctic; Climate change; Marine mammals; Seabir...,CLIMATE-CHANGE; BEAUFORT SEA; SEABIRDS; SHIPS;...,The Arctic is changing rapidly due to climate ...,"[Halliday, William D.] Wildlife Conservat Soc ...",NaN,"Halliday, WD (corresponding author), Wildlife ...",...,1,1,ELSEVIER SCI LTD,1462-9011,1873-6416,2022.0,10.1016/j.envsci.2021.10.026,http://dx.doi.org/10.1016/j.envsci.2021.10.026,"Green Published, hybrid",NaN
4,"Samhouri, JF; Feist, BE; Fisher, MC; Liu, O; W...","Samhouri, Jameal F.; Feist, Blake E.; Fisher, ...",Marine heatwave challenges solutions to human-...,PROCEEDINGS OF THE ROYAL SOCIETY B-BIOLOGICAL ...,trade-offs; dynamic ocean management; Dungenes...,ECOSYSTEM SERVICES; DYNAMIC OCEAN; WEST-COAST;...,Despite the increasing frequency and magnitude...,"[Samhouri, Jameal F.; Feist, Blake E.; Fisher,...",NaN,"Samhouri, JF (corresponding author), NOAA, Con...",...,4,4,ROYAL SOC,0962-8452,1471-2954,2021.0,10.1098/rspb.2021.1607,http://dx.doi.org/10.1098/rspb.2021.1607,"Green Published, hybrid",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,"Wang, RL; Jiang, CX; Huang, TT; Zhang, Z; Wang...","Wang, Rulin; Jiang, Chunxian; Huang, Tingting;...",A Simulation Study of the Geographical Distrib...,POLISH JOURNAL OF ENVIRONMENTAL STUDIES,Actinidia arguta; MaxEnt; environmental factor...,CLIMATE-CHANGE; POTENTIAL DISTRIBUTION; MAXENT...,The aim of this study was to conduct an ecolog...,"[Wang, Rulin; Jiang, Chunxian; Huang, Tingting...",NaN,"Li, Q (corresponding author), Sichuan Agr Univ...",...,5,5,HARD,NaN,NaN,2020.0,10.15244/pjoes/109306,http://dx.doi.org/10.15244/pjoes/109306,gold,NaN
49996,"Shapouri, MRSA; Mahmoodi, P; Najafabadi, MG; H...","Shapouri, Masoud Reza Seyfi Abad; Mahmoodi, Pe...",A novel competitive ELISA for detection of ant...,VETERINARY RESEARCH FORUM,Bovine viral diarrhea; Competit

In [51]:
#This will make a new column in the dataframe with the first author's country
df['First_A_Country']=first_author_country

In [52]:
df

,Authors,Author Full Names,Article Title,Source Title,Author Keywords,Keywords Plus,Abstract,Addresses,Affiliations,Reprint Addresses,...,"Times Cited, All Databases",Publisher,ISSN,eISSN,Publication Year,DOI,DOI Link,Open Access Designations,Highly Cited Status,First_A_Country
0,"Glidden, CK; Field, LC; Bachhuber, S; Hennesse...","Glidden, Caroline K.; Field, Laurel C.; Bachhu...",Strategies for managing marine disease,ECOLOGICAL APPLICATIONS,disease ecology; marine conservation; marine w...,ACUTE RESPIRATORY SYNDROME; ENDANGERED WHITE A...,The incidence of emerging infectious diseases ...,"[Glidden, Caroline K.; Field, Laurel C.; Bachh...",NaN,"Gravem, SA (corresponding author), Oregon Stat...",...,1,WILEY,1051-0761,1939-5582,2022.0,10.1002/eap.2643,http://dx.doi.org/10.1002/eap.2643,"Green Submitted, hybrid",NaN,USA
1,"Avery-Gomm, S; Borrelle, SB; Provencher, JF","Avery-Gomm, Stephanie; Borrelle, Stephanie B.;...",Linking plastic ingestion research with marine...,SCIENCE OF THE TOTAL ENVIRONMENT,Plastic pollution; Conservation; Policy; Waste...,DEBRIS; MANAGEMENT; SEABIRDS; MICROPLASTICS; S...,Plastic is an increasingly pervasive marine po...,"[Avery-Gomm, Stephanie] Univ Queensland, Sch B...",NaN,"Avery-Gomm, S (corresponding author), Univ Que...",...,26,ELSEVIER SCIENCE BV,0048-9697,1879-1026,2018.0,10.1016/j.scitotenv.2018.04.409,http://dx.doi.org/10.1016/j.scitotenv.2018.04.409,Green Submitted,NaN,Australia
2,"Guerra, AS","Guerra, Ana Sofia",Wolves of the Sea: Managing human-wildlife con...,MARINE POLICY,Competition; Depredation; Human-wildlife confl...,CONSERVATION; MARINE; DEPREDATION; FISHERIES; ...,Human-wildlife conflict has been receiving inc...,"[Guerra, Ana Sofia] Univ Calif Santa Barbara, ...",NaN,"Guerra, AS (corresponding author), Univ Calif ...",...,29,ELSEVIER SCI LTD,0308-597X,1872-9460,2019.0,10.1016/j.marpol.2018.11.002,http://dx.doi.org/10.1016/j.marpol.2018.11.002,NaN,NaN,USA
3,"Halliday, WD; Dawson, J; Yurkowski, DJ; Doniol...","Halliday, William D.; Dawson, Jackie; Yurkowsk...",Vessel risks to marine wildlife in the Talluru...,ENVIRONMENTAL SCIENCE & POLICY,Arctic; Climate change; Marine mammals; Seabir...,CLIMATE-CHANGE; BEAUFORT SEA; SEABIRDS; SHIPS;...,The Arctic is changing rapidly due to climate ...,"[Halliday, William D.] Wildlife Conservat Soc ...",NaN,"Halliday, WD (corresponding author), Wildlife ...",...,1,ELSEVIER SCI LTD,1462-9011,1873-6416,2022.0,10.1016/j.envsci.2021.10.026,http://dx.doi.org/10.1016/j.envsci.2021.10.026,"Green Published, hybrid",NaN,Canada
4,"Samhouri, JF; Feist, BE; Fisher, MC; Liu, O; W...","Samhouri, Jameal F.; Feist, Blake E.; Fisher, ...",Marine heatwave challenges solutions to human-...,PROCEEDINGS OF THE ROYAL SOCIETY B-BIOLOGICAL ...,trade-offs; dynamic ocean management; Dungenes...,ECOSYSTEM SERVICES; DYNAMIC OCEAN; WEST-COAST;...,Despite the increasing frequency and magnitude...,"[Samhouri, Jameal F.; Feist, Blake E.; Fisher,...",NaN,"Samhouri, JF (corresponding author), NOAA, Con...",...,4,ROYAL SOC,0962-8452,1471-2954,2021.0,10.1098/rspb.2021.1607,http://dx.doi.org/10.1098/rspb.2021.1607,"Green Published, hybrid",NaN,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,"Wang, RL; Jiang, CX; Huang, TT; Zhang, Z; Wang...","Wang, Rulin; Jiang, Chunxian; Huang, Tingting;...",A Simulation Study of the Geographical Distrib...,POLISH JOURNAL OF ENVIRONMENTAL STUDIES,Actinidia arguta; MaxEnt; environmental factor...,CLIMATE-CHANGE; POTENTIAL DISTRIBUTION; MAXENT...,The aim of this study was to conduct an ecolog...,"[Wang, Rulin; Jiang, Chunxian; Huang, Tingting...",NaN,"Li, Q (corresponding author), Sichuan Agr Univ...",...,5,HARD,NaN,NaN,2020.0,10.15244/pjoes/109306,http://dx.doi.org/10.15244/pjoes/109306,gold,NaN,Peoples R China
49996,"Shapouri, MRSA; Mahmoodi, P; Najafabadi, MG; H...","Shapouri, Masoud Reza Seyfi Abad; Mahmoodi, Pe...",A novel competitive ELISA for detection of ant...,VETERINARY RESEARCH FORUM,Bovine v

In [53]:
df.to_csv('Combined_WoS_Dataset.csv')

## All geographic data has been transferred, and is ready to be divided for sampling